In [7]:
# Importing necessary libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Checking available device (CPU/GPU)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Defining data transformations

In [9]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Loading dataset and Providing the path to the already separatef training, validation and testing sets

In [10]:
train_dataset = torchvision.datasets.ImageFolder(root=r'/kaggle/input/wildfire-comp-6721/archive/train', transform=data_transforms)
valid_dataset = torchvision.datasets.ImageFolder(root=r'/kaggle/input/wildfire-comp-6721/archive/valid', transform=data_transforms)
test_dataset = torchvision.datasets.ImageFolder(root=r'/kaggle/input/wildfire-comp-6721/archive/test', transform=data_transforms)

# Generate a random sample of 10 images from the test dataset


In [ ]:
# Generate a random sample of 10 images from the test dataset
sample_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=10, shuffle=True)
sample_iter = iter(sample_loader)
sample_images, sample_labels = next(sample_iter)

# Display the sample images
plt.figure(figsize=(15, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(sample_images[i].permute(1, 2, 0))  # Transpose the tensor to (H, W, C) for display
    plt.title(f"Class: {sample_labels[i]}")
    plt.axis('off')
plt.show()


# Batch size and learning rate configurations


In [12]:
batch_size = 32
learning_rate = 0.0001
num_epochs = 40

# Print the configuration
print(f"Training with batch size: {batch_size}, learning rate: {learning_rate}, and epochs: {num_epochs}")


Training with batch size: 32, learning rate: 0.0001, and epochs: 40


# Creating data loaders


In [13]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initializing AlexNet model


In [14]:
model = models.vgg16(pretrained=False)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2)
model.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Defining loss function and optimizer


In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# Lists to store loss and accuracy during training


In [16]:
loss_during_epoch = []
accuracy_during_epoch = []


# Training loop


In [17]:
patience = 5
best_valid_loss = float('inf')
epochs_no_improve = 0
early_stopping = False

train_loss_list = []
train_accuracy_list = []
valid_loss_list = []
valid_accuracy_list = []

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for inputs, labels in train_loader:
        # Your existing training code...
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)


    # Validation
    model.eval()
    valid_loss = 0.0
    valid_correct_predictions = 0
    valid_total_predictions = 0

    with torch.no_grad():
        for valid_inputs, valid_labels in valid_loader:
            valid_inputs, valid_labels = valid_inputs.to(device), valid_labels.to(device)
            valid_outputs = model(valid_inputs)
            valid_loss += criterion(valid_outputs, valid_labels).item()

            _, valid_predicted = torch.max(valid_outputs, 1)
            valid_correct_predictions += (valid_predicted == valid_labels).sum().item()
            valid_total_predictions += valid_labels.size(0)


    end_time = time.time()
    elapsed_time = end_time - start_time

    # Calculate metrics
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct_predictions / total_predictions
    valid_loss = valid_loss / len(valid_loader)
    valid_accuracy = valid_correct_predictions / valid_total_predictions

    # Append to lists
    train_loss_list.append(train_loss)
    train_accuracy_list.append(train_accuracy)
    valid_loss_list.append(valid_loss)
    valid_accuracy_list.append(valid_accuracy)

    print(f"Epoch {epoch+1}, Loss: {train_loss}, Accuracy: {train_accuracy * 100:.2f}%, "
          f"Validation Loss: {valid_loss}, Validation Accuracy: {valid_accuracy * 100:.2f}%, "
          f"Running time: {elapsed_time}")

    # Early stopping logic
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            print("Early stopping!")
            early_stopping = True
            break

# After the loop, you can check if early stopping occurred
if early_stopping:
    print("Training stopped due to early stopping.")
else:
    print("Training completed all epochs.")

Epoch 1, Loss: 0.21602039487736002, Accuracy: 91.65%, Validation Loss: 0.19438252762565153, Validation Accuracy: 92.49%, Running time: 305.10466146469116
Epoch 2, Loss: 0.1247875734411359, Accuracy: 95.95%, Validation Loss: 0.2041352304067406, Validation Accuracy: 91.52%, Running time: 252.42972993850708
Epoch 3, Loss: 0.09697922782318012, Accuracy: 96.70%, Validation Loss: 0.2561862035489022, Validation Accuracy: 91.24%, Running time: 252.90551376342773
Epoch 4, Loss: 0.07648856920440467, Accuracy: 97.36%, Validation Loss: 0.28654638075548683, Validation Accuracy: 90.41%, Running time: 251.32966828346252
Epoch 5, Loss: 0.08066607334451084, Accuracy: 97.37%, Validation Loss: 0.24074674152821152, Validation Accuracy: 91.43%, Running time: 254.2286355495453
Epoch 6, Loss: 0.06332987209421222, Accuracy: 97.86%, Validation Loss: 0.29125752916020636, Validation Accuracy: 91.40%, Running time: 253.65048789978027
Early stopping!
Training stopped due to early stopping.


# Saving the model weights


In [18]:
torch.save(model.state_dict(), f'Wildfire_ResNet18_weights_batch_size_{batch_size}_lr_{learning_rate}_epochs_{num_epochs}.pth')


# Evaluation on test set


In [19]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())


# Calculating evaluation metrics


In [20]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')
f1 = f1_score(y_true, y_pred, average='micro')
cm = confusion_matrix(y_true, y_pred)


# Printing evaluation metrics



In [21]:
print(f"Batch Size: {batch_size}, Learning Rate: {learning_rate}, Epochs: {num_epochs}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Batch Size: 32, Learning Rate: 0.0001, Epochs: 40
Accuracy: 0.9395171537484117
Precision: 0.9395171537484117
Recall: 0.9395171537484117
F1 Score: 0.9395171537484117


# Plotting confusion matrix


Confusion Matrix:


In [22]:
print("Confusion Matrix:")
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[f'Class {i}' for i in range(len(cm))], yticklabels=[f'Class {i}' for i in range(len(cm))])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Plotting accuracy and loss during training


In [ ]:
# Determine the minimum length between range and train_loss_list
min_length = min(len(range(1, 18 + 1)), len(train_loss_list))

# Plotting training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, min_length + 1), train_loss_list[:min_length], label='Training Loss')
plt.plot(range(1, min_length + 1), valid_loss_list[:min_length], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Determine the minimum length between range and train_accuracy_list
min_length_acc = min(len(range(1, 18 + 1)), len(train_accuracy_list))

# Plotting training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(range(1, min_length_acc + 1), train_accuracy_list[:min_length_acc], label='Training Accuracy')
plt.plot(range(1, min_length_acc + 1), valid_accuracy_list[:min_length_acc], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()